In [2]:
import pandas as pd
from pandas import DataFrame as DF, Series
import numpy as np
import requests
import re

In [3]:
client_token = '230875850816746|hh1JJh8ebiRPuFb_V85xqto4lNo'
def search(q):
    url = 'https://graph.facebook.com/v2.12/search?'
    params = {'access_token': client_token,
              'q': q,
              'type': 'place',
              'fields': 'category_list,about,website,name,location'}
    r = requests.get(url, params=params).json()['data']
    locations = []
    results = []
    for d in r:
        for k in list(d):
            if k == 'category_list':
                d[k] = ','.join([_['name'] for _ in d[k]])
            elif k == 'location':
                _ = d[k].copy()
                _['id'] = d['id']
                locations.append(_)
                del d[k], _
        results.append(d)
    return DF(results).merge(DF(locations), on='id')

## Transaction Detail First Batch

In [6]:
import io

url = """https://raw.githubusercontent.com/hackoregon/elections-2018/
master/transaction_detail_first_batch.csv"""
r = requests.get(url.replace('\n', ''))
xdf = pd.read_csv(io.StringIO(r.content.decode('utf-8')), dtype={'Agent': str})

In [7]:
xdf.iloc[40]

IndexError: single positional indexer is out-of-bounds

In [8]:
# some business names & addresses
xdf[xdf.loc[:, 'Address Book Type'] == 'Business Entity']\
    .loc[:, ['Name', 'Address']]\
    .drop_duplicates().head(20)

KeyError: 'the label [Address Book Type] is not in the [columns]'

In [13]:
df = search("Nickeklf")

KeyError: 'id'

In [14]:
df.head().T

,0,1,2,3,4
about,NaN,NaN,NaN,NaN,We are greatful to be part of helping to be sp...
category_list,Footwear Store,Sporting Goods Store,Clothing Store,High School,"Sporting Goods Store,Region,Sports Club"
id,195780813847551,618778561628746,111979372161808,250138881817377,120450974981872
name,Nike,Nike,Nike shoes,"Shiksha Niketan School, Jeevan Nagar",Nike team
website,http://www.nike.com,http://www.nike.com/es/es_es/c/countries/unite...,nike.com,http://www.shikshaniketanjk.in/,http://m.nike.com/us/en_us/
city,Portland,London,Beaverton,Jammu,Portland
country,United States,United Kingdom,United States,India,United States
latitude,45.519,51.5157,45.5093,32.6796,45.5418
located_in,154528042042,NaN,1507288099571823,NaN,NaN
longitude,-122.677,-0.141792,-122.83,74.9043,-122.661


In [15]:
search('Oregon Speech Language Hearing Association').head().T

,0
about,The Oregon Speech-Language-Hearing Association...
category_list,"Speech Pathologist,Organization"
id,142750555806480
name,Oregon Speech-Language & Hearing Association
website,http://www.oregonspeechandhearing.org
city,Salem
country,United States
latitude,44.9376
located_in,105362302832944
longitude,-123.025


In [9]:
search('Loaves & Fishes Centers, Inc.').head().T

,0
about,Beckley Dream Center is a Faith Based Non-Prof...
category_list,Nonprofit Organization
id,627133924085198
name,Beckley Dream Center INC DBA Fishes and Loaves
website,http://beckleydreamcenter.wix.com/beckley-drea...
city,Beckley
country,United States
latitude,37.8092
longitude,-81.1891
state,WV


In [1]:
df = search('National Multiple Sclerosis Society')
df.head().T

NameError: name 'search' is not defined

In [11]:
def matches(x, ex):
    print(x, ex)
    if x != x:
        return False
    if re.search(x, ex):
        return True
    else:
        return False
    
addr = xdf[xdf.Name == 'National Multiple Sclerosis Society'].Address.tolist()[0]
addr = 'SW Clay'
df[df.apply(lambda x: matches(x['street'], addr), axis=1)]

2440 S Sepulveda Blvd, Ste 115 SW Clay
2211 W Meadowview Rd SW Clay
104 SW Clay St SW Clay
nan SW Clay
192 Nickerson St, Ste 100 SW Clay
900 S Broadway, Ste 250 SW Clay
1669 W Avenue J SW Clay
5025 E Washington St SW Clay
14 W Valerio St SW Clay
4221 Northgate Blvd SW Clay
1 Reed St, Ste 200 SW Clay
214 Overlook Cir, Ste 153 SW Clay
205 Hallene Rd, Ste 209 SW Clay
1630 Manheim Pike SW Clay
101 1st Ave, Ste 6 SW Clay
2105 Luna Rd, Ste 390 SW Clay
5 E Citrus Ave Ste 107 SW Clay
8111 N Stadium Dr SW Clay
nan SW Clay
9600 N Mo Pac Expy SW Clay
2589 Scott Blvd SW Clay
2209 Forest Hill Drive, Suite 18 SW Clay
2112 W Laburnum Ave SW Clay
1700 Owens St SW Clay
525 W. Monroe St., Ste. 1510 SW Clay


,about,category_list,id,name,website,city,country,latitude,located_in,longitude,state,street,zip


In [14]:
df.apply(lambda x: matches(x['street'], addr), axis=1)

2440 S Sepulveda Blvd, Ste 115 SW Clay
2211 W Meadowview Rd SW Clay
104 SW Clay St SW Clay
nan SW Clay
192 Nickerson St, Ste 100 SW Clay
900 S Broadway, Ste 250 SW Clay
1669 W Avenue J SW Clay
5025 E Washington St SW Clay
14 W Valerio St SW Clay
4221 Northgate Blvd SW Clay
1 Reed St, Ste 200 SW Clay
214 Overlook Cir, Ste 153 SW Clay
205 Hallene Rd, Ste 209 SW Clay
1630 Manheim Pike SW Clay
101 1st Ave, Ste 6 SW Clay
2105 Luna Rd, Ste 390 SW Clay
5 E Citrus Ave Ste 107 SW Clay
8111 N Stadium Dr SW Clay
nan SW Clay
9600 N Mo Pac Expy SW Clay
2589 Scott Blvd SW Clay
2209 Forest Hill Drive, Suite 18 SW Clay
2112 W Laburnum Ave SW Clay
1700 Owens St SW Clay
525 W. Monroe St., Ste. 1510 SW Clay


0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
dtype: bool

In [15]:
addr

'SW Clay'

In [16]:
df.iloc[14]

about            Since 1950, the Greater New England Chapter ha...
category_list    Charity Organization,Nonprofit Organization,Pu...
id                                                    390667451084
name              National MS Society, Greater New England Chapter
website                                http://www.msnewengland.org
city                                                       Waltham
country                                              United States
latitude                                                   42.3936
located_in                                                     NaN
longitude                                                 -71.2691
state                                                           MA
street                                          101 1st Ave, Ste 6
zip                                                          02451
Name: 14, dtype: object